<a href="https://colab.research.google.com/github/iammuhammad41/Medical-Image-Segmentation/blob/main/medical_segmentation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --progress-bar on -q evalseg==2.0.8

# for installing latest version from github
# !pip uninstall -y evalseg
# !pip install -U --progress-bar on  git+https://github.com/modaresimr/evalseg.git

In [ ]:
import evalseg
import numpy as np
import pandas as pd
from ipywidgets import interact, IntSlider
import os

In [ ]:
root_data='/kaggle/input/medical-image-segmentation/'

'''# Ref. Medical Image Segmentation: Evaluation: https://www.kaggle.com/dsv/4728423'''

In [2]:
@interact
def select_dataset(dataset_name=sorted([d for d in os.listdir(root_data) if os.path.isdir(f'{root_data}/{d}')])):
    global dataset

    dataset = evalseg.io.Dataset(f'{root_data}/{dataset_name}')

    @interact
    def select_case(case=sorted(dataset.get_available_ids())):

        gto= dataset.get_groundtruth(case)
        cto = dataset.get_CT(case)

        @interact
        def ax_select(ax=['z', 'x', 'y', 'all']):
            gto_dense=gto.todense()
            gtmax = (gto_dense.sum(axis=2).sum(axis=1).argmax(), gto_dense.sum(axis=2).sum(axis=0).argmax(), gto_dense.sum(axis=1).sum(axis=0).argmax())
            axi = {'all': -1, 'x': 0, 'y': 1, 'z': 2}[ax]

            @interact
            def onlyMaxFrame(frame=IntSlider(-1, min=-1, max=-1) if ax == 'all' else IntSlider(gtmax[axi], min=0, max=gto.shape[axi]-1)):
                global data, gt, voxelsize
                evalseg.reload()
                np.seterr(all='raise')

                gt = evalseg.geometry.slice_segment(gto, axi, [frame])
                ct = evalseg.geometry.slice_segment(cto, axi, [frame])
                mme = evalseg.metrics.MultiClassMetric(
                    evalseg.metrics.MME(debug={'U': 1, 'B': 1, 'T':1,'R':1,'D': 1, 'UI': 1}),
                    dataset.num_labels,
                    debug={'log': 1}
                )
                mme.set_reference(gt)

                @interact
                def pred_selec(prediction=dataset.get_prediction_methods(case)):
                    global pred
                    pred = dataset.get_prediction(prediction, case)
                    pred = evalseg.geometry.slice_segment(pred,  axi, [frame])
                    if ax != 'all':
                        ctlbl = f"{dataset_name.split('_')[1]} {case} {ax}={frame}"
                        comps = mme.metrics[1].helper['components']
                        print(comps)
                        gt2 = gt.todense()
                        for c in comps:
                            gt2 = gt2 & ~ comps[c]['gt_skeleton'].todense()
                        evalseg.ui.multi_plot_2d(ct.todense(), gt2, {prediction: pred.todense()}, spacing=gt.voxelsize, ctlbl=ctlbl, z_titles=[frame], crop2roi=0)
                    else:
                        pass  # TODO
                    global debug_info
                    res,debug_info = mme.evaluate(pred, parallel=0,return_debug=True)
                    evalseg.ui.plot_metric(res, name=dataset_name)
